In [166]:
## ALGORITMO VECINO MÁS PRÓXIMO ##

In [167]:
## ABRIR DATASET ACTUALIZADO
import pickle

with open("dataset_10_actualizado.pkl", "rb") as f:
    datos = pickle.load(f)

#print(datos);print(datos.keys());print(datos["data"]);print(datos["distancias"]);print(datos["tiempos"]);print(datos["costes"])


In [168]:
## CREAR EL CONJUNTO DE NODOS ##
base_operativa_ficticia=-1
N = [i for i,d in enumerate(datos["data"]) if d['nodo']=="carga"] # nodos de carga
F = [i for i,d in enumerate(datos["data"]) if d['nodo']=="cargador"] # nodos cargador
V = list(range(len(datos["data"])))+[base_operativa] # Nodos totales
base_operativa = [i for i,d in enumerate(datos["data"]) if d['nodo']=="parking"][0] 
descarga = [i for i,d in enumerate(datos["data"]) if d['nodo']=="descarga"][0] 

#print(V);print(N);print(F);print(base_operativa);print(descarga);print(base_operativa)


In [169]:
# CREACIÓN DE ARRAYS
import numpy as np
c_0 = np.array(datos["costes"])
w_0 = np.array([d["w"] for d in datos["data"]])
tiempos_0 = np.array(datos["tiempos"])
distancias=np.array(datos["distancias"])

In [170]:
# La idea es elegir los cargadores por la distancia y luego sumarles el precio de carga. Pero como tenemos en cuenta esa diferencia de precio para la elección? Pues lo tomamos
# como si el coste de llegar a esos cargadores fuese, para los de menos coste, solo su coste real, y para los de más coste, su coste real + la diferencia de precio de carga.

# Vamos a actualizar el vector de costes.
# Tomaremos para los cargadores de ionity su coste real de llegar (por ser el mínimo)
# Tomaremos para los cargadores de Atlante la diferencia (9) de precio de carga + el coste total
c=[[0]*len(c_0) for _ in range(len(c_0))]
for i in range(len(c_0)):
    for j in range(len(c_0)):
        if i==F[2] or i==F[3]: #cargadores de Atlante
            c[i][j]=c_0[i][j]+9
            if j!=i:
                c[j][i]=c_0[j][i]+9
            else: 
                c[j][i]=c_0[j][i]
        else:
            c[i][j]=c_0[i][j]
c = np.array(c)

#for filas in c:
#    print(filas)

[ 0.     6.656 14.748  9.628  7.268  5.816  2.368  3.976 14.192 16.044
  5.132]
[ 6.656  0.    21.096 14.672  8.592 11.148  4.584  9.212 18.152 12.504
  9.584]
[14.748 21.096  0.    15.136 15.616 14.324 16.52  14.808 25.508 30.74
 12.184]
[ 9.628 14.672 15.136  0.    16.544  3.816 11.764  5.748 14.544 21.708
 13.22 ]
[ 7.268  8.592 15.616 16.544  0.    12.832  6.12  11.168 21.452 20.396
  3.988]
[ 5.816 11.148 14.324  3.816 12.832  0.     7.96   1.98  11.564 18.676
  9.732]
[ 2.368  4.584 16.52  11.764  6.12   7.96   0.     6.016 15.808 14.932
  5.44 ]
[ 3.976  9.212 14.808  5.748 11.168  1.98   6.016  0.    10.836 16.992
  8.4  ]
[14.192 18.152 25.508 14.544 21.452 11.564 15.808 10.836  0.    16.26
 19.004]
[16.044 12.504 30.74  21.708 20.396 18.676 14.932 16.992 16.26   0.
 20.372]
[ 5.132  9.584 12.184 13.22   3.988  9.732  5.44   8.4   10.004 11.372
  0.   ]


In [172]:
# ALGORITMO DEL VECINO MÁS PRÓXIMO 
no_visitados = N.copy()+ F.copy()
cargadores= F.copy()  
ruta = [base_operativa] # La ruta comienza en la base operativa
posicion = base_operativa # Posición actual del vehículo
coste_total = 0
costes= c.copy()

while no_visitados:
    costes_posibles = {i: costes[posicion][i] for i in no_visitados}
    if not costes_posibles:
        break 

    # Encontrar el coste mínimo
    cost = min(costes_posibles.values())
    siguiente = min(costes_posibles, key=costes_posibles.get)

    # Si se selecciona un cargador, no se visitan más cargadores
    if siguiente in cargadores:
        no_visitados= [i for i in no_visitados if i not in cargadores]
        cost+= 183 # coste de  carga (a los de atlante ya se le sumó 9) 
    else: 
          no_visitados.remove(siguiente)

    # Actualizar la ruta
    coste_total += cost
    ruta.append(siguiente)

    # Actualizar la posición
    posicion = siguiente

# Ir a la descarga y volver a la base operativa
ruta.append(descarga)
coste_total += costes[posicion][descarga] 
ruta.append(base_operativa)
coste_total += costes[descarga][base_operativa] 
print("Ruta:",ruta) 
print("Coste de la distancia recorrida:", coste_total)


Ruta: [0, 6, 1, 4, 2, 3, 5, 0]
Coste de la distancia recorrida: 238.92800000000003


In [173]:

# La ruta que hemos hecho no tiene en cuenta si se puede ir a un cargador entre la descarga y la base operativa
# Vamos a calcular esa ruta y ver si es mejor que la que tenemos
no_visitados_1 = N.copy()
ruta_1 = [base_operativa] # La ruta comienza en la base operativa
posicion_1 = base_operativa # Posición actual del vehículo
coste_total_1 = 0
costes_1= c.copy()

while no_visitados_1:
    costes_posibles_1 = {i: costes_1[posicion_1][i] for i in no_visitados_1}
    if not costes_posibles_1:
        break 

    # Encontrar el coste mínimo
    cost_1 = min(costes_posibles_1.values())
    siguiente_1 = min(costes_posibles_1, key=costes_posibles_1.get)

    # Actualizar la ruta
    coste_total_1 += cost_1
    ruta_1.append(siguiente_1)
    no_visitados_1.remove(siguiente_1)

    # Actualizar la posición
    posicion_1 = siguiente_1

# Ir a la descarga 
ruta_1.append(descarga)
coste_total_1 += costes_1[posicion_1][descarga] 

# Visitar un cargador
cargadores_1= F.copy()
costes_cargadores={i: costes_1[descarga][i] for i in cargadores_1}

# Encontrar el coste mínimo
cost_2 = min(costes_cargadores.values())
siguiente_1 = min(costes_cargadores, key=costes_cargadores.get)
coste_total_1 += cost_2 + 183
ruta_1.append(siguiente_1)

#Volver a la base operativa
ruta_1.append(base_operativa)
coste_total_1 += costes_1[descarga][base_operativa] 
print("Ruta:",ruta_1) 
print("Coste de la distancia recorrida:", coste_total_1)




Ruta: [0, 1, 4, 2, 3, 5, 7, 0]
Coste de la distancia recorrida: 240.612


In [174]:
rutas={tuple(ruta):coste_total, tuple(ruta_1):coste_total_1}
ruta_optima = min(rutas, key=rutas.get)
print("Ruta óptima:", ruta_optima)
print("Coste mínimo:", rutas[ruta_optima])

Ruta óptima: (0, 6, 1, 4, 2, 3, 5, 0)
Coste mínimo: 238.92800000000003
